<a href="https://colab.research.google.com/github/luizvmachado/calculadora_aluguel/blob/main/Web_Scrap_VivaReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
list_titulo_anuncio = list()
list_endereco_anuncio = list()
list_area_anuncio = list()
list_numero_quartos = list()
list_numero_banheiros = list()
list_numero_garagem = list()
list_preco = list()
list_condominio = list()
endereco = list()
numeros = list()
bairro =list()
cidade = list()
estado = list()

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.maximize_window()

In [ ]:
pbar = tqdm(range(1,200))
for pagina in pbar:
    url = f'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina={pagina}'
    pbar.set_description("Processando página %s" % pagina)
    driver.get(url)
    resultados = driver.find_elements(By.CLASS_NAME,'property-card__content')
    pbar2 = tqdm(resultados)
    for elemento_1 in pbar2:
        list_titulo_anuncio.append(elemento_1.find_element(By.CLASS_NAME,'property-card__title').text)
        list_endereco_anuncio.append(elemento_1.find_element(By.CLASS_NAME,'property-card__address-container').text)
        list_area_anuncio.append(elemento_1.find_element(By.CLASS_NAME,'property-card__detail-area').text)
        list_numero_quartos.append(elemento_1.find_element(By.CLASS_NAME,'property-card__detail-room').text)
        list_numero_banheiros.append(elemento_1.find_element(By.CLASS_NAME,'property-card__detail-bathroom').text)
        list_numero_garagem.append(elemento_1.find_element(By.CLASS_NAME,'property-card__detail-garage').text)
        list_preco.append(elemento_1.find_element(By.CLASS_NAME,'property-card__price').text)       
        try:
            list_condominio.append(elemento_1.find_element(By.CLASS_NAME,'property-card__price-details--condo').text)
        except:
            list_condominio.append('N/A')    

In [ ]:
for end in tqdm(list_endereco_anuncio):
    tokens_macro = end.split("-")    
    endereco_numero = tokens_macro[0].split(",")    
    if len(endereco_numero)>1:
        endereco.append(endereco_numero[0].strip())
        numeros.append(str(endereco_numero[1].strip()))
    else:
        endereco.append(endereco_numero[0].strip())
        numeros.append('0')      
    try:
        b, c = tokens_macro[1].split(',')
        bairro.append(b.strip())
        cidade.append(c.strip())
    except:
        bairro.append('N/A')
        cidade.append('N/A')
        continue
areas = [float(area.split(" ")[0]) for area in list_area_anuncio]
quartos = [int(q.split(" ")[0]) for q in list_numero_quartos]
banheiros = [int(b.split(" ")[0]) for b in list_numero_banheiros]
vagas_garagem = [int(g.split(" ")[0]) if g != '-- Vaga' else 0 for g in list_numero_garagem]
precos = [float(p.split(" ")[1].replace(".","")) for p in list_preco]
condominio = [float(c.split(" ")[-1].replace(".","")) if c!= "N/A" else 0.0 for c in list_condominio]

In [ ]:
dataset = pd.DataFrame({'Titulo': list_titulo_anuncio,
                        'Endereco': endereco,
                        'Numero': numeros,
                        'Bairro': bairro,
                        'Cidade': cidade,
                        'Area': areas,
                        'Quartos':quartos,
                        'Banheiros':banheiros,
                        'Vagas':vagas_garagem,
                        'Preco':precos})

In [ ]:
dataset.head()

In [ ]:
dataset.to_excel('ScrapVivaReal.xlsx', index = False)
dataset.to_csv('ScrapVivaRealCSV.xlsx', sep = ',', index = False)